

# Analyse setiment twitter \newline avec pyspark et mlib 



In [1]:
sc



<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [3]:
data=spark.read.json("hdfs://localhost:9000/sparkData/data.txt")

In [4]:
positiveW=spark.read.text("hdfs://localhost:9000/sparkData/positive-words.txt")
positiveW.show()


+---------------+
|          value|
+---------------+
|             a+|
|         abound|
|        abounds|
|      abundance|
|       abundant|
|     accessable|
|     accessible|
|        acclaim|
|      acclaimed|
|    acclamation|
|       accolade|
|      accolades|
|  accommodative|
|   accomodative|
|     accomplish|
|   accomplished|
| accomplishment|
|accomplishments|
|       accurate|
|     accurately|
+---------------+
only showing top 20 rows



In [8]:
negativeW=spark.read.text("hdfs://localhost:9000/sparkData/negative-words.txt")
#negativeW=sc.textFile("hdfs://localhost:9000/sparkData/negative-words.txt")



In [9]:
sql=spark.sql("SELECT id,text FROM id_tweet")
#sql.show()

In [11]:
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType

s=sql.withColumn("sentiment",lit(0)+5)
s.describe()

r=s.withColumn("TEST",s["sentiment"].cast(IntegerType())-51)
#r.show()
s.schema
                
    

StructType(List(StructField(id,LongType,true),StructField(text,StringType,true),StructField(sentiment,IntegerType,false)))

In [23]:
datardd=s.rdd
datardd.collect()

[Row(id=330043883738234880, text="'Iron Man 3' Crushes Opening Day Records In China: \nIron Man 3 has continued its record-breaking run at the... http://t.co/9CwF31E8Ez", sentiment=5),
 Row(id=330043886577799169, text='Iron Man 3 tonight! Woo :)', sentiment=5),
 Row(id=330043887055937536, text='Preciso ver Iron Man 3 todos comentando e eu boiando kk', sentiment=5),
 Row(id=330043888788197376, text="'Iron Man 3' Crushes Opening Day Records In China: \nIron Man 3 has continued its record-breaking run at the... http://t.co/sAX3MsJjlE", sentiment=5),
 Row(id=330043889589288960, text='Iron man 3 was awesome =)', sentiment=5),
 Row(id=330043890440749057, text='RT @HannahMenzies: Featuring @mattfraction on The Beat today from Iron Man 50th panel!\nhttp://t.co/qHNJMKjNl9', sentiment=5),
 Row(id=330043896786726913, text='RT @puthatasstowork: ce soir, go to look Iron Man 3! :D', sentiment=5),
 Row(id=330043897516531714, text='Iron man 3 with the homies! pfernando805 sglende lzepeda75 http://t.co

### negativewords to rdd because we cant proform map on dataframe then flatmap the rdd rows to arraysso we can get arrays of negative and positive words

In [15]:
negativeW.select('value').take(2)

[Row(value='2-faced'), Row(value='2-faces')]

In [16]:
negativeW.select('value').rdd.take(2)

[Row(value='2-faced'), Row(value='2-faces')]

In [17]:
negativeW.select('value').rdd.flatMap(lambda x:x).take(4)

['2-faced', '2-faces', 'abnormal', 'abolish']

In [18]:
negativeList=negativeW.select('value').rdd.flatMap(lambda x:x)

In [19]:
positiveList=positiveW.select("value").rdd.flatMap(lambda x:x)

In [20]:
negativeW.select('value').rdd.flatMap(lambda x:x).map(lambda x:(x,1)).take(3)

[('2-faced', 1), ('2-faces', 1), ('abnormal', 1)]

### lets try to d this with out main dataframe datardd=s.rdd (s is our data id,text,sentiment)

In [22]:
q=["good", "gold", "silver"]
z="hey bith gold good"

In [23]:
"good" in q

True

### we split each text row into an array of words

In [24]:
textarrays=s.select("text").rdd.map(lambda x: x[0].split())
textarrays.take(2)


[["'Iron",
  'Man',
  "3'",
  'Crushes',
  'Opening',
  'Day',
  'Records',
  'In',
  'China:',
  'Iron',
  'Man',
  '3',
  'has',
  'continued',
  'its',
  'record-breaking',
  'run',
  'at',
  'the...',
  'http://t.co/9CwF31E8Ez'],
 ['Iron', 'Man', '3', 'tonight!', 'Woo', ':)']]

### define de finction that counts negative/positive words in an array and score it

In [25]:
tnls=negativeList.collect()
tpls=positiveList.collect()


In [26]:
def find(Textarray):
    count=0
    for j in range(len(Textarray)):
            if Textarray[j] in tpls:
                count=count+1
            elif Textarray[j] in tnls:
                count=count-1   
    if count>0:
        return("Positive Tweet")
    elif count<0:
        return("Negative Tweet")
    else:
        return("neutral Tweet")


In [27]:
z=["'Iron",
  'Man',
  "3'",
  'Crushes',
  'Opening',
  'Day',
  'Records',
  'In',
  'China:',
  'Iron',
  'Man',
  '3',
  'has',
  'continued',
  'its',
  'record-breaking',
  'run',
  'at',
  'the...',
  'http://t.co/9CwF31E8Ez',
  'abound',
  'abort',
  'aborted']
#z[1].find("Man")
#"Man" in z[1]
#find(z,"Man")
find (z)

'Negative Tweet'

#### here we will find how many times input x(array of words aka tweet words occur in nega words list)

In [28]:
textarrays.map(lambda x: (find(x),x)).countByKey()
#textarrays.map(lambda x: type(x)).collect()
#textarrays.map(lambda x: filter(lambda y:"hina" in y, )).take(10)
#textarrays.map(lambda x: ((1 if find("China") in x else 0),x)).take(3)

defaultdict(int,
            {'neutral Tweet': 1568,
             'Positive Tweet': 289,
             'Negative Tweet': 160})

In [29]:
#s.select("text").rdd.map(lambda x:(1 if any(word in x for word in q),))

now we can find words in a text cell , we will convert 

In [30]:
textarrays.map(lambda x: (find(x),x)).countByKey()

defaultdict(int,
            {'neutral Tweet': 1568,
             'Positive Tweet': 289,
             'Negative Tweet': 160})

In [31]:
s.schema

StructType(List(StructField(id,LongType,true),StructField(text,StringType,true),StructField(sentiment,IntegerType,false)))

### here we take x row of datardd : x= id, text,sentiment , then we apply a map function to get the same output
### but we apply find method on x["text"] after spliting it (bc it was a string now its an array of words) to get the score if negative or positive

In [32]:
newdf=datardd.map(lambda x:(find(x["text"].split()),x["text"],x["id"]  )).toDF()

we rename the cols after transforming rdd to df

In [33]:
lll=newdf.toDF("Sentiment","TweetText","Id")
lll.show(2)
    

+-------------+--------------------+------------------+
|    Sentiment|           TweetText|                Id|
+-------------+--------------------+------------------+
|neutral Tweet|'Iron Man 3' Crus...|330043883738234880|
|neutral Tweet|Iron Man 3 tonigh...|330043886577799169|
+-------------+--------------------+------------------+
only showing top 2 rows



we join on key=id our new df with the old data

In [34]:
datawithsentiment=data.join(lll,on="id").drop("TweetText")
#datawithsentiment=data.join(lll,on="id").drop("TweetText").printSchema()
datawithsentiment.show(2)

+------------------+------------+-----------+--------------------+--------------------+--------------+---------+------------+----+------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+----+-----+------------------+-------------+---------+----------------+--------------------+--------------------+---------+--------------------+-------------+
|                id|contributors|coordinates|          created_at|            entities|favorite_count|favorited|filter_level| geo|            id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|lang|place|possibly_sensitive|retweet_count|retweeted|retweeted_status|              source|                text|truncated|                user|    Sentiment|
+------------------+------------+-----------+--------------------+--------------------+--------------+---------+------------+----+------------------+---

sooo we have data mixed some with sentiment others without aka neutral

In [35]:
datawithsentiment.groupBy('Sentiment').count().show()     #select('Sentiment').count().show()

+--------------+-----+
|     Sentiment|count|
+--------------+-----+
|Negative Tweet|  160|
|Positive Tweet|  289|
| neutral Tweet| 1568|
+--------------+-----+



In [36]:
datawithsentiment.filter((datawithsentiment.Sentiment!="neutral Tweet")).groupBy('Sentiment').count().show() 

+--------------+-----+
|     Sentiment|count|
+--------------+-----+
|Negative Tweet|  160|
|Positive Tweet|  289|
+--------------+-----+



In [37]:
dfwithsentolny=datawithsentiment.filter((datawithsentiment.Sentiment!="neutral Tweet"))

## lets try to do some classification now lets use the three sentiment compleete dataframe


### first lets encode the sentiment column -1 negative / 0 neutral / 1 positive

In [38]:
from pyspark.sql.functions import col, when

encodedData=datawithsentiment.withColumn("Sentiment", when(col("Sentiment")=='Negative Tweet', -1).when(col("Sentiment")=='Positive Tweet', 1).otherwise(0))

In [39]:
(train_set, val_set, test_set) = encodedData.randomSplit([0.5, 0.3, 0.2], seed = 2000)

In [40]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "Sentiment", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])


In [51]:
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
test_df=pipelineFit.transform(test_set)
#train_df.select("text","Sentiment","words","tf","features","label").show(10)
test_df.groupBy('label').count().show() 

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  302|
|  1.0|   63|
|  2.0|   29|
+-----+-----+



In [42]:
train_df.select("text","Sentiment","words","tf","features","label").show(3)

+--------------------+---------+--------------------+--------------------+--------------------+-----+
|                text|Sentiment|               words|                  tf|            features|label|
+--------------------+---------+--------------------+--------------------+--------------------+-----+
|'Iron Man 3' Crus...|        0|['iron, man, 3', ...|(65536,[2460,9960...|(65536,[2460,9960...|  0.0|
|Iron Man 3 tonigh...|        0|[iron, man, 3, to...|(65536,[23538,249...|(65536,[23538,249...|  0.0|
|Preciso ver Iron ...|        0|[preciso, ver, ir...|(65536,[3885,1206...|(65536,[3885,1206...|  0.0|
+--------------------+---------+--------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [49]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)
type(lrModel)

pyspark.ml.classification.LogisticRegressionModel

In [50]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.6451430897016115

In [192]:
type(predictions)

pyspark.sql.dataframe.DataFrame

In [45]:
#val_df.select('label','Sentiment').take(10)
#val_df.filter(val_df.Sentiment==val_df.label).count()/float(val_set.count())
predictions.filter(predictions.prediction==predictions.label).count()/float(predictions.count())

0.802675585284281

Here we see how the predictions are made when the validation df is transformed

In [56]:
predictions.select("Sentiment","label","rawPrediction","probability","prediction").take(10)

[Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([30.8011, 1.1316, -31.9327]), probability=DenseVector([1.0, 0.0, 0.0]), prediction=0.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([67.1321, -15.6295, -51.5026]), probability=DenseVector([1.0, 0.0, 0.0]), prediction=0.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([43.346, -36.6131, -6.7329]), probability=DenseVector([1.0, 0.0, 0.0]), prediction=0.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([-5.156, -17.7639, 22.9199]), probability=DenseVector([0.0, 0.0, 1.0]), prediction=2.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([30.8011, 1.1316, -31.9327]), probability=DenseVector([1.0, 0.0, 0.0]), prediction=0.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([30.8011, 1.1316, -31.9327]), probability=DenseVector([1.0, 0.0, 0.0]), prediction=0.0),
 Row(Sentiment=0, label=0.0, rawPrediction=DenseVector([44.0976, -61.1138, 17.0162]), probability=DenseVector([1.0, 0.0, 0.0]), predic

### lets evaluate our test data

### we take randomly a row of our test dataset then we tranform it and evaluate it (we can choose the sample size)

In [158]:
from pyspark.sql.functions import rand
testingSample=test_df.select("*").orderBy(rand())
testsamplesized=testingSample.drop("label").limit(100)
testsamplesizedwithlabel=testingSample.limit(100)

In [123]:
#test_df.createOrReplaceTempView("test_df")
#query=spark.sql("SELECT * FROM test_df ORDERBY RAND() LIMIT 1 ")

### we will predict the one without labels

In [159]:
predictSample=lrModel.transform(testsamplesized)

binaryclassifEvaluator not gonna use it

In [156]:
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
#evaluator.evaluate(predictSample)


my eveluator (number of correct preds / total )

In [160]:
predictSample.filter(predictSample.prediction==testsamplesizedwithlabel.label).count()/float(predictSample.count())

0.78

In [162]:
predictSample.select("Sentiment","prediction","rawPrediction","probability").take(5)

[Row(Sentiment=0, prediction=0.0, rawPrediction=DenseVector([6.1398, 0.8288, -6.9686]), probability=DenseVector([0.9951, 0.0049, 0.0])),
 Row(Sentiment=1, prediction=1.0, rawPrediction=DenseVector([-69.0674, 39.2189, 29.8485]), probability=DenseVector([0.0, 0.9999, 0.0001])),
 Row(Sentiment=1, prediction=1.0, rawPrediction=DenseVector([16.5213, 45.3412, -61.8625]), probability=DenseVector([0.0, 1.0, 0.0])),
 Row(Sentiment=0, prediction=1.0, rawPrediction=DenseVector([14.4244, 16.5408, -30.9652]), probability=DenseVector([0.1075, 0.8925, 0.0])),
 Row(Sentiment=0, prediction=0.0, rawPrediction=DenseVector([19.928, -1.6947, -18.2332]), probability=DenseVector([1.0, 0.0, 0.0]))]